In [51]:
import pandas as pd
import numpy as np

import torch as th
import torch.nn.functional as fn

from sklearn import metrics
from sklearn import preprocessing
import matplotlib.pyplot as plt

In [52]:
def transE(head, rel, tail, gamma=12.0):
    # Paper link: https://papers.nips.cc/paper/5071-translating-embeddings-for-modeling-multi-relational-data
    score = head + rel - tail
    
    return gamma - th.norm(score, p=2, dim=-1)

In [53]:
def transR(head, rel, tail, proj, rel_idx, gamma=12.0):
    # Paper link: https://www.aaai.org/ocs/index.php/AAAI/AAAI15/paper/download/9571/9523
    proj = proj.reshape(-1, head.shape[1], rel.shape[0])[rel_idx]
    head_r = th.einsum('ab,bc->ac', head, proj)
    tail_r = th.einsum('b,bc->c', th.tensor(tail), proj)
    score = head_r + rel - tail_r
    
    return gamma - th.norm(score, p=1, dim=-1)

In [54]:
def DistMult(head, rel, tail):
    # Paper link: https://arxiv.org/abs/1412.6575
    score = head * rel * tail
    
    return th.sum(score, dim=-1)

In [55]:
def complEx(head, rel, tail, gamma=12.0):
    # Paper link: https://arxiv.org/abs/1606.06357
    real_head, img_head = th.chunk(head, 2, dim=-1)
    real_tail, img_tail = th.chunk(th.tensor(tail), 2, dim=-1)
    real_rel, img_rel = th.chunk(rel, 2, dim=-1)

    score = real_head * real_tail * real_rel \
            + img_head * img_tail * real_rel \
            + real_head * img_tail * img_rel \
            - img_head * real_tail * img_rel

    return th.sum(score, -1)

In [56]:
Drug_list_folder = 'MLH/Drug_list_ad/'
kg_folder = ''
result_folder = 'MLH/ADResults/'
folder = 'MLH/'

In [57]:
def AD_drugs_possibility_prediction(model_name, trial_status):
    entity_df = pd.read_table(folder + '/entities.tsv', header=None)##
    entity_df = entity_df.dropna().reset_index(drop=True)
    approved_drug_df = pd.read_csv(Drug_list_folder + 'drugs_list_' + trial_status + '.csv')## approved_drug_df = pd.read_csv(Drug_list_folder + 'drugs_list_' + trial_status + '.csv')
    approved_drug_list = list(approved_drug_df['Drug'])

    entity_map = {}
    entity_id_map = {}
    relation_map = {}
    drug_ids = []
    drug_names = []
    disease_ids = []

    for i in range(len(entity_df)):
        entity_id = entity_df.loc[i, 0]
        entity_name = entity_df.loc[i, 1]
        entity_map[entity_name] = int(entity_id)
        entity_id_map[int(entity_id)] = entity_name
        if entity_name.replace('DrugBank:', '') in approved_drug_list:
            drug_ids.append(entity_id)
            drug_names.append(entity_name.replace('DrugBank:', ''))

    disease_vocab = pd.read_csv(kg_folder + 'Entity/disease_vocab.csv')
    AD_related_list = [] ##
    for i in range(len(disease_vocab)):
        primary_id = disease_vocab.loc[i, 'primary']
        disease_name = disease_vocab.loc[i, 'name']
        disease_name = disease_name if not pd.isnull(disease_name) else ''
        if 'alzheimer' in disease_name:
            if primary_id not in AD_related_list:
                AD_related_list.append(primary_id)

    relation_df = pd.read_table(folder + '/relations.tsv', header=None)
    for i in range(len(relation_df)):
        relation_id = relation_df.loc[i, 0]
        relation_name = relation_df.loc[i, 1]
        relation_map[relation_name] = int(relation_id)

    for disease in AD_related_list:##
        if disease in entity_map:
            disease_ids.append(entity_map[disease])

    entity_emb = np.load(folder + model_name + '/iBKH_' + model_name + '_entity.npy')
    rel_emb = np.load(folder + model_name + '/iBKH_' + model_name + '_relation.npy')
    if model_name == 'TransR':
        proj_np = np.load(folder + 'TransR/iBKH_TransRprojection.npy')
        proj_emb = th.tensor(proj_np)

    treatment = ['Treats_DDi', 'Palliates_DDi', 'Effect_DDi', 'Associate_DDi', 'Inferred_Relation_DDi',
                 'Semantic_Relation_DDi']
    treatment_rid = [relation_map[treat] for treat in treatment]

    drug_ids = th.tensor(drug_ids).long()
    disease_ids = th.tensor(disease_ids).long()
    treatment_rid = th.tensor(treatment_rid)

    drug_emb = th.tensor(entity_emb[drug_ids])
    treatment_embs = [th.tensor(rel_emb[rid]) for rid in treatment_rid]

    scores_per_disease = []
    dids = []
    for rid in range(len(treatment_embs)):
        treatment_emb = treatment_embs[rid]
        for disease_id in disease_ids:
            disease_emb = th.tensor(entity_emb[disease_id])
            if model_name == 'RotatE':
                score = fn.logsigmoid(rotatE(drug_emb, treatment_emb, disease_emb))
            elif model_name == 'ComplEx':
                score = fn.logsigmoid(complEx(drug_emb, treatment_emb, disease_emb))
            elif model_name == 'TransR':
                score = fn.logsigmoid(transR(drug_emb, treatment_emb, disease_emb, proj_emb, treatment_rid[rid]))
            elif model_name == 'TransE':
                score = fn.logsigmoid(transE(drug_emb, treatment_emb, disease_emb))
            elif model_name == 'DistMult':
                score = fn.logsigmoid(DistMult(drug_emb, treatment_emb, disease_emb))
            scores_per_disease.append(score)
            dids.append(drug_ids)
    #print(scores_per_disease)
    scores = th.cat(scores_per_disease)
    dids = th.cat(dids)

    idx = th.flip(th.argsort(scores), dims=[0])
    scores = scores[idx].numpy()
    dids = dids[idx].numpy()

    _, unique_indices = np.unique(dids, return_index=True)
    topk_indices = np.sort(unique_indices)
    proposed_dids = dids[topk_indices]
    proposed_scores = scores[topk_indices]

    candidate_drug_rank = []
    candidate_drug_score = {}
    for i, idx in enumerate(proposed_dids):
        candidate_drug_rank.append(entity_id_map[int(idx)].replace('DrugBank:', ''))
        candidate_drug_score[entity_id_map[int(idx)].replace('DrugBank:', '')] = proposed_scores[i]

    df = pd.DataFrame(columns=['Drug', 'Score'])
    idx = 0
    for drug in candidate_drug_score:
        df.loc[idx] = [drug, candidate_drug_score[drug]]
        idx += 1

    x = np.asarray(df['Score']).reshape(-1, 1)  # returns a numpy array
    min_max_scaler = preprocessing.MinMaxScaler()
    x_scaled = min_max_scaler.fit_transform(x)
    df['Score_scaled'] = pd.DataFrame(x_scaled)
    #print(df)
    df.to_csv(result_folder + "predict_result_scaled_" + model_name + "_" + trial_status + ".csv", index=False)

In [58]:
def ensemble_model(trial_status):
    transE_res = pd.read_csv(result_folder + "predict_result_scaled_TransE_" + trial_status + ".csv")['Drug'].tolist()
    transR_res = pd.read_csv(result_folder + "predict_result_scaled_TransR_" + trial_status + ".csv")['Drug'].tolist()
    complEx_res = pd.read_csv(result_folder + "predict_result_scaled_ComplEx_" + trial_status + ".csv")['Drug'].tolist()
    distMult_res = pd.read_csv(result_folder + "predict_result_scaled_DistMult_" + trial_status + ".csv")['Drug'].tolist()

    res = pd.DataFrame(columns=['Drug', 'vote_score'])
    idx = 0
    for drug in transE_res:
        vote_transE = len(transE_res) - transE_res.index(drug)
        vote_transR = len(transR_res) - transR_res.index(drug)
        vote_complEx = len(complEx_res) - complEx_res.index(drug)
        vote_distMult = len(distMult_res) - distMult_res.index(drug)
        vote_score = vote_transE + vote_transR + vote_complEx + vote_distMult
        res.loc[idx] = [drug, vote_score]
        idx += 1
    res = res.sort_values('vote_score', ascending=False)
    res = res.reset_index(drop=True)
    x = np.asarray(res['vote_score']).reshape(-1, 1)
    min_max_scaler = preprocessing.MinMaxScaler()
    x_scaled = min_max_scaler.fit_transform(x)
    res['Score_scaled'] = pd.DataFrame(x_scaled)
    #print(res)
    res.to_csv(result_folder + "predict_result_scaled_ensemble_" + trial_status + ".csv", index=False)

In [59]:
def generate_AUC(model_name):
    figures_folder = result_folder + 'roc_figures/'
    drug_trial_list = ['approve_phase1234', 'approve_phase234', 'approve_phase34', 'approve_phase4', 'approve']
    trial_info = {'approve_phase1234': {'label': 'FDA approved+Phase I~IV', 'color': '#6a4c93'},
                  'approve_phase234': {'label': 'FDA approved+Phase II~IV', 'color': '#1982c4'},
                  'approve_phase34': {'label': 'FDA approved+Phase III,IV', 'color': '#8ac926'},
                  'approve_phase4': {'label': 'FDA approved+Phase IV', 'color': '#ffca3a'},
                  'approve': {'label': 'FDA approved', 'color': '#ff595e'}}
    plt.figure(figsize=(7, 7))

    for trial_status in drug_trial_list:
        predict_res = pd.read_csv(result_folder + "predict_result_scaled_" + model_name + "_" + trial_status + ".csv")
        candidate_df = pd.read_csv(Drug_list_folder + 'drugs_list_' + trial_status + '.csv')##

        df = pd.merge(predict_res, candidate_df, on='Drug')

        label = np.array(list(df['label']))
        score = np.array(list(df['Score_scaled']))
        #score = np.array(list(df['Score']))
        #print("Label:", label, "Score:", score)
        fpr, tpr, thresholds = metrics.roc_curve(label, score)
        youden = tpr - fpr
        youden_J = np.max(youden)
        inds_youden_J = np.where(youden == youden_J)
        #print("Inds Youden J:" ,inds_youden_J)
        tpr_max = tpr[inds_youden_J]
        fpr_max = fpr[inds_youden_J]
        cut_off =  thresholds[inds_youden_J][0]
        sensitivity = tpr_max[0]
        specificity = 1 - fpr_max[0]
        prevalence = np.where(label == 1)[0].shape[0] / label.shape[0]
        acc = (sensitivity * prevalence) + (specificity * (1 - prevalence))
        print(cut_off, sensitivity, specificity, acc)
        auc = metrics.auc(fpr, tpr)
        plt.plot(fpr, tpr, label=trial_info[trial_status]['label'] + ', AUC=' + str(round(auc, 2)),
                 color=trial_info[trial_status]['color'])

    plt.plot([0, 1], [0, 1], linestyle='--', lw=2, color='gray', alpha=.8)
    plt.tick_params(labelsize=16, bottom=True, left=True)
    plt.xlabel("1 - Specificity", fontsize=12, fontweight='bold')
    plt.ylabel("Sensitivity", fontsize=12, fontweight='bold')
    plt.grid(alpha=.3)
    plt.legend(prop={'size': 12}, loc=4)
    plt.title(model_name, fontweight='bold', fontsize=18)
    # plt.show()
    plt.savefig(figures_folder + model_name + '.jpg', dpi=300)
    plt.close()

In [60]:
# we try different strategies to build the ground truth PARKINSONS drug list

drug_trial_list = [
    'approve_phase1234', # FDA Approved AD drugs + AD drugs in clinical trials Phase I, II, III and IV. 
    'approve_phase234',  # FDA Approved AD drugs + AD drugs in clinical trials Phase II, III and IV. 
    'approve_phase34',   # FDA Approved AD drugs + AD drugs in clinical trials Phase III and IV. 
    'approve_phase4',    # FDA Approved AD drugs + AD drugs in clinical trials Phase IV. 
    'approve'            # FDA Approved AD drugs only. 
]

### Could be a batchsize issue with the model. Have to ask fabio to potentially scale down the drug size list. 
Fundamental error with the dataset, the way its designed wont work. 

In [62]:
model_name = 'TransE'
for trial_status in drug_trial_list:
    AD_drugs_possibility_prediction(model_name, trial_status)

generate_AUC(model_name)

0.90850186 0.5531914893617021 0.6642780365460408 0.657267539442766
0.92376256 0.45454545454545453 0.7564785232516862 0.740828004039044
0.92376256 0.625 0.7543798177995795 0.7508520790729378
0.92376256 0.7435897435897436 0.7500866551126517 0.7500000000000001
0.93467414 0.7777777777777778 0.8147512864493996 0.8146374829001368


In [63]:
model_name = 'TransR'
for trial_status in drug_trial_list:
    AD_drugs_possibility_prediction(model_name, trial_status)

generate_AUC(model_name)

C:\Users\hruda\AppData\Local\Temp\ipykernel_6280\632631403.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  tail_r = th.einsum('b,bc->c', th.tensor(tail), proj)
C:\Users\hruda\AppData\Local\Temp\ipykernel_6280\632631403.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  tail_r = th.einsum('b,bc->c', th.tensor(tail), proj)
C:\Users\hruda\AppData\Local\Temp\ipykernel_6280\632631403.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  tail_r = th.einsum('b,bc->c', th.tensor(tail), proj)
C:\Users\hruda\AppData\Local\Temp\ipykernel_6280\632

0.9959231 0.3829787234042553 0.8491580078824794 0.8197381671701913
0.9959231 0.37662337662337664 0.845580404685836 0.8212722988892627
0.9868466 0.7 0.6800981079187105 0.6806407634628493
0.98716235 0.8205128205128205 0.6797227036395148 0.6816005471956225
0.99418056 0.7777777777777778 0.851114922813036 0.8508891928864569


In [64]:
model_name = 'DistMult'
for trial_status in drug_trial_list:
    AD_drugs_possibility_prediction(model_name, trial_status)

generate_AUC(model_name)

0.7106764 0.39361702127659576 0.7972053027588678 0.7717354817052703
0.7106764 0.38961038961038963 0.7941072062477813 0.7731403567822281
0.77733016 0.4625 0.8472319551506657 0.8367416496250852
0.60831016 0.717948717948718 0.7261698440207972 0.7260601915184678
0.4568792 0.7777777777777778 0.6435677530017152 0.6439808481532147


In [65]:
model_name = 'ComplEx'
for trial_status in drug_trial_list:
    AD_drugs_possibility_prediction(model_name, trial_status)

generate_AUC(model_name)

C:\Users\hruda\AppData\Local\Temp\ipykernel_6280\697845562.py:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  real_tail, img_tail = th.chunk(th.tensor(tail), 2, dim=-1)
C:\Users\hruda\AppData\Local\Temp\ipykernel_6280\697845562.py:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  real_tail, img_tail = th.chunk(th.tensor(tail), 2, dim=-1)
C:\Users\hruda\AppData\Local\Temp\ipykernel_6280\697845562.py:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  real_tail, img_tail = th.chunk(th.tensor(tail), 2, dim=-1)
C:\Users\hruda\AppData\Local\Temp\

0.87013674 0.5 0.6897169473307059 0.6777442094662639
0.8709707 0.4935064935064935 0.6886758963436279 0.6785594076068663
0.87883323 0.6375 0.7102312543798178 0.7082481254260395
0.8794366 0.7435897435897436 0.7077989601386482 0.7082763337893297
0.87964684 0.8888888888888888 0.7060034305317324 0.7065663474692202


In [66]:
for trial_status in drug_trial_list:
    ensemble_model(trial_status)

generate_AUC('ensemble')

0.7790990365419669 0.4308510638297872 0.8093873163740595 0.7854984894259819
0.7967118997912318 0.4025974025974026 0.8239261625843095 0.8020868394479972
0.8086282199839558 0.5375 0.8405746320953048 0.8323108384458077
0.8080672569537608 0.6666666666666666 0.836048526863085 0.8337893296853626
0.8080672569537608 0.7777777777777778 0.8312178387650085 0.8310533515731874
